# Merkmalsstandardisierung in PCA: HR-Analytics Beispiel

## Das Geschäftsproblem

Du bist HR-Analyst in einem großen Unternehmen mit **25+ Mitarbeiterkennzahlen**. Dein Ziel ist es:
- 🎯 **Mitarbeitersegmente identifizieren** für gezielte HR-Strategien
- 💰 **Vergütungsentscheidungen optimieren**
- 📈 **Leistung und Kündigungsrisiken vorhersagen**
- 🔍 **Komplexität reduzieren** von dutzenden korrelierten Variablen

**Die Herausforderung**: Mit 25+ Variablen wird traditionelle Analyse unmöglich. PCA kann helfen, die Dimensionalität zu reduzieren und gleichzeitig wichtige Muster zu bewahren.

## Lernziele

- Verstehen, warum PCA für hochdimensionale Geschäftsdaten wertvoll ist
- Sehen, wie verschiedene Skalen PCA-Ergebnisse völlig verzerren können
- Lernen, wann und warum Merkmale standardisiert werden sollten
- Üben, Hauptkomponenten im Geschäftskontext zu interpretieren

## Vorbereitung

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Stil für bessere Plots setzen
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("🚀 Bereit, HR-Daten mit PCA zu analysieren!")

## 1. Die Realität: Komplexer HR-Datensatz

Zuerst, lass uns sehen, wie ein **echter HR-Analytics-Datensatz** mit 25+ Variablen aussieht:

In [ ]:
# Generiere realistischen HR-Datensatz mit 25+ Variablen
def generiere_komplexe_hr_daten(n_mitarbeiter=1000):
    """Generiere realistischen HR-Datensatz mit 25+ korrelierten Variablen"""
    
    # Basis-Faktoren
    erfahrung = np.random.gamma(3, 2, n_mitarbeiter)
    bildung = np.random.choice([1, 2, 3, 4], n_mitarbeiter, p=[0.1, 0.3, 0.4, 0.2])
    
    # Leistung (korreliert mit Erfahrung und Bildung)
    leistung_basis = 2.5 + 0.3 * bildung + 0.1 * erfahrung + np.random.normal(0, 0.5, n_mitarbeiter)
    leistung = np.clip(leistung_basis, 1, 5)
    
    # Gehalt (stark korreliert mit Leistung, Erfahrung, Bildung)
    gehalt_basis = 35000 + 15000 * bildung + 2000 * erfahrung + 8000 * leistung
    gehalt = gehalt_basis + np.random.normal(0, 5000, n_mitarbeiter)
    gehalt = np.clip(gehalt, 30000, 200000)
    
    # Viele weitere Variablen...
    daten = pd.DataFrame({
        'Mitarbeiter_ID': range(1, n_mitarbeiter + 1),
        'Jahresgehalt': gehalt.round(0),
        'Leistungsbewertung': leistung.round(2),
        'Jahre_Erfahrung': erfahrung.round(1),
        'Bildungsgrad': bildung,
        'Projekte_Abgeschlossen': np.random.poisson(5 + leistung, n_mitarbeiter),
        'Ueberstunden_Monatlich': np.random.gamma(2, 10, n_mitarbeiter).round(1),
        'Schulungsstunden_Jaehrlich': (20 + 10 * bildung + np.random.exponential(15, n_mitarbeiter)).round(1),
        'Arbeitszufriedenheit': np.clip(3 + 0.00002 * (gehalt - 50000) + 0.3 * leistung + np.random.normal(0, 0.8, n_mitarbeiter), 1, 5).round(2),
        'Teamgroesse': np.random.choice([3, 5, 8, 12, 20], n_mitarbeiter, p=[0.2, 0.3, 0.3, 0.15, 0.05]),
        'Kundenzufriedenheit': np.clip(3.5 + 0.4 * leistung + np.random.normal(0, 0.6, n_mitarbeiter), 1, 5).round(2),
        'Krankheitstage_Letztes_Jahr': np.random.poisson(3, n_mitarbeiter),
        'Innovationen_Vorgeschlagen': np.random.poisson(1 + leistung, n_mitarbeiter),
        'Fuehrungsbewertung': np.clip(2 + 0.4 * erfahrung + 0.3 * leistung + np.random.normal(0, 0.7, n_mitarbeiter), 1, 5).round(2),
        'Technische_Faehigkeiten': np.clip(2 + bildung * 0.5 + np.random.normal(0, 0.8, n_mitarbeiter), 1, 5).round(2),
        'Kommunikationsbewertung': np.clip(3 + 0.4 * leistung + np.random.normal(0, 0.6, n_mitarbeiter), 1, 5).round(2),
        'Work_Life_Balance': np.clip(3 + np.random.normal(0, 1, n_mitarbeiter), 1, 5).round(2),
        'Karriereentwicklung': np.clip(2.5 + 0.3 * leistung + np.random.normal(0, 0.8, n_mitarbeiter), 1, 5).round(2),
        'Meeting_Teilnahme': np.clip(0.7 + 0.2 * (leistung / 5) + np.random.normal(0, 0.1, n_mitarbeiter), 0.3, 1.0).round(3),
        'Email_Antwortzeit_Stunden': np.random.exponential(2, n_mitarbeiter).round(1),
        'Kollaborationsbewertung': np.clip(3 + 0.3 * leistung + np.random.normal(0, 0.5, n_mitarbeiter), 1, 5).round(2),
        'Problemloesungsfaehigkeit': np.clip(2.5 + 0.5 * bildung + np.random.normal(0, 0.7, n_mitarbeiter), 1, 5).round(2),
        'Mentoring_Stunden': np.random.gamma(1, 5, n_mitarbeiter).round(1),
        'Prozessverbesserungen': np.random.poisson(2 + leistung * 0.5, n_mitarbeiter),
        'Alter': np.clip(22 + erfahrung + np.random.normal(0, 2, n_mitarbeiter), 22, 65).round(0)
    })
    
    return daten

# Generiere komplexen Datensatz
komplexe_daten = generiere_komplexe_hr_daten(1000)

print("🏢 KOMPLEXER HR-DATENSATZ")
print("=" * 50)
print(f"📊 Datensatz-Form: {komplexe_daten.shape}")
print(f"📈 Anzahl Variablen: {komplexe_daten.shape[1] - 1} (ohne Mitarbeiter_ID)")
print("\n👥 Stichprobe von Mitarbeitern:")
print(komplexe_daten.head())

In [ ]:
# Zeige die Komplexitätsherausforderung
print("🤯 DIE KOMPLEXITÄTSHERAUSFORDERUNG:")
print("=" * 45)

# Zeige Korrelationen
numerische_spalten = komplexe_daten.select_dtypes(include=[np.number]).columns.drop('Mitarbeiter_ID')
korrelationen = komplexe_daten[numerische_spalten].corr()

print(f"📊 Mit {len(numerische_spalten)} Variablen haben wir:")
print(f"   • {len(numerische_spalten) * (len(numerische_spalten) - 1) // 2} einzigartige Korrelationen zu analysieren")
print(f"   • Unmöglich alle Beziehungen zu visualisieren")
print(f"   • Schwierig Muster manuell zu identifizieren")

print("\n🔗 Top-Korrelationen mit Leistungsbewertung:")
leistung_korr = korrelationen['Leistungsbewertung'].sort_values(ascending=False)
print(leistung_korr.head(8))

print("\n💡 Genau deshalb brauchen wir PCA!")
print("   PCA kann diese 24 Variablen auf 2-3 bedeutungsvolle Komponenten reduzieren")

In [ ]:
# Visualisiere die Korrelationsmatrix
plt.figure(figsize=(14, 12))
maske = np.triu(np.ones_like(korrelationen, dtype=bool))
sns.heatmap(korrelationen, mask=maske, annot=False, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .5})
plt.title('HR-Daten Korrelationsmatrix\n(24 Variablen = 276 einzigartige Korrelationen!)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("😵‍💫 Überwältigend, oder? PCA wird uns helfen, das zu verstehen!")

## 2. Vereinfachtes Lehrbeispiel

Für Lernzwecke konzentrieren wir uns auf **zwei Schlüsselvariablen**, die das Standardisierungskonzept klar demonstrieren:
- **Jahresgehalt** (30.000€ - 200.000€)
- **Leistungsbewertung** (1,0 - 5,0)

> **Hinweis**: In echten Projekten würdest du alle 24+ Variablen verwenden. Wir verwenden hier 2 Variablen, um klar zu sehen, was PCA macht und warum Standardisierung wichtig ist.

In [ ]:
# Extrahiere unsere Lehrvariablen und erstelle einen fokussierten Datensatz
# Wir wählen Mitarbeiter aus, die ein interessantes Muster zeigen

lehrdaten = np.array([
    [45000, 4.8],   # Aufsteiger: Niedriges Gehalt, hohe Leistung
    [48000, 4.9],   # Aufsteiger
    [52000, 4.7],   # Aufsteiger
    [47000, 4.6],   # Aufsteiger
    [49000, 4.8],   # Aufsteiger
    [46000, 4.9],   # Aufsteiger
    [55000, 4.4],   # Aufsteiger
    [95000, 2.1],   # Überbezahlt: Hohes Gehalt, niedrige Leistung
    [98000, 2.3],   # Überbezahlt
    [88000, 2.5],   # Überbezahlt
    [92000, 2.2],   # Überbezahlt
    [85000, 2.6],   # Überbezahlt
    [90000, 2.4],   # Überbezahlt
    [65000, 3.5],   # Ausgewogen: Mittleres Gehalt, mittlere Leistung
    [70000, 3.2],   # Ausgewogen
    [68000, 3.4],   # Ausgewogen
    [72000, 3.3],   # Ausgewogen
])

df = pd.DataFrame(lehrdaten, columns=['Jahresgehalt', 'Leistungsbewertung'])
df['Mitarbeiter_ID'] = range(1, len(df) + 1)

print("💼 HR-LEHRDATENSATZ")
print("=" * 35)
print("Ausgewählte Mitarbeiter mit klaren Mustern:")
print(df)
print(f"\n📊 Datensatz-Form: {df[['Jahresgehalt', 'Leistungsbewertung']].shape}")

In [ ]:
# Analysiere die Datencharakteristika
print("📈 DATENCHARAKTERISTIKA:")
print("=" * 35)
print(df[['Jahresgehalt', 'Leistungsbewertung']].describe())

# Zeige den Skalenunterschied
gehalt_bereich = df['Jahresgehalt'].max() - df['Jahresgehalt'].min()
leistung_bereich = df['Leistungsbewertung'].max() - df['Leistungsbewertung'].min()
skalenverhaeltnis = df['Jahresgehalt'].std() / df['Leistungsbewertung'].std()

print(f"\n⚖️ SKALENANALYSE:")
print(f"Gehalt-Bereich: {df['Jahresgehalt'].min():,.0f}€ - {df['Jahresgehalt'].max():,.0f}€")
print(f"Leistungs-Bereich: {df['Leistungsbewertung'].min():.1f} - {df['Leistungsbewertung'].max():.1f}")
print(f"Standardabweichungs-Verhältnis: {skalenverhaeltnis:.0f}:1")
print(f"💡 Gehaltswerte sind {skalenverhaeltnis:.0f}x variabler als Leistungswerte!")

# Prüfe Korrelation
korrelation = df['Jahresgehalt'].corr(df['Leistungsbewertung'])
print(f"\n🔗 Korrelation: {korrelation:.3f}")
if korrelation < -0.3:
    print("📉 Negative Korrelation deutet auf überbezahlte Leistungsschwache hin!")
elif korrelation > 0.3:
    print("📈 Positive Korrelation deutet auf Leistungsbezahlung hin!")
else:
    print("➡️ Schwache Korrelation deutet auf gemischte Vergütungsmuster hin")

## 3. Daten visualisieren

In [ ]:
# Erstelle umfassende Visualisierung
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Streudiagramm der Rohdaten
axes[0,0].scatter(df['Jahresgehalt'], df['Leistungsbewertung'], s=100, alpha=0.7, c='steelblue')
axes[0,0].set_xlabel('Jahresgehalt (€)')
axes[0,0].set_ylabel('Leistungsbewertung (1-5)')
axes[0,0].set_title('HR-Daten: Gehalt vs Leistung\n(Beachte den Skalenunterschied!)')
axes[0,0].grid(True, alpha=0.3)

# Füge Anmerkungen für Mitarbeitertypen hinzu
axes[0,0].annotate('Aufsteiger\n(Niedriges Gehalt, hohe Leistung)', 
                   xy=(48000, 4.8), xytext=(55000, 4.5),
                   arrowprops=dict(arrowstyle='->', color='green'),
                   fontsize=10, color='green', weight='bold')
axes[0,0].annotate('Überbezahlt?\n(Hohes Gehalt, niedrige Leistung)', 
                   xy=(95000, 2.2), xytext=(85000, 3.0),
                   arrowprops=dict(arrowstyle='->', color='red'),
                   fontsize=10, color='red', weight='bold')

# Plot 2: Gehaltsverteilung
axes[0,1].hist(df['Jahresgehalt'], bins=10, alpha=0.7, color='orange', edgecolor='black')
axes[0,1].set_xlabel('Jahresgehalt (€)')
axes[0,1].set_ylabel('Anzahl Mitarbeiter')
axes[0,1].set_title('Gehaltsverteilung')
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Leistungsverteilung
axes[1,0].hist(df['Leistungsbewertung'], bins=10, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1,0].set_xlabel('Leistungsbewertung (1-5)')
axes[1,0].set_ylabel('Anzahl Mitarbeiter')
axes[1,0].set_title('Leistungsverteilung')
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Skalenvergleich
merkmale = ['Gehalt\n(45K€-98K€)', 'Leistung\n(2,1-4,9)']
std_abw = [df['Jahresgehalt'].std(), df['Leistungsbewertung'].std()]
axes[1,1].bar(merkmale, std_abw, color=['orange', 'lightcoral'], alpha=0.7)
axes[1,1].set_ylabel('Standardabweichung')
axes[1,1].set_title('Skalenunterschied-Problem')
axes[1,1].set_yscale('log')  # Log-Skala um den dramatischen Unterschied zu zeigen

# Füge Wertbeschriftungen hinzu
for i, v in enumerate(std_abw):
    axes[1,1].text(i, v * 1.1, f'{v:.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"🎯 Die Gehalts-Standardabweichung ({df['Jahresgehalt'].std():.0f}) ist {skalenverhaeltnis:.0f}x größer!")
print("Dieser Skalenunterschied wird PCA dominieren, wenn wir nicht standardisieren.")

## 4. PCA ohne Standardisierung

Lass uns sehen, was passiert, wenn wir PCA direkt auf die Rohdaten anwenden:

In [ ]:
# Bereite Daten für PCA vor
X = df[['Jahresgehalt', 'Leistungsbewertung']].values

# Wende PCA ohne Standardisierung an
pca_roh = PCA()
X_pca_roh = pca_roh.fit_transform(X)

print("🚫 PCA OHNE STANDARDISIERUNG")
print("=" * 45)
print(f"📊 Erklärtes Varianzverhältnis: {pca_roh.explained_variance_ratio_}")
print(f"📈 Kumulative Varianz: {np.cumsum(pca_roh.explained_variance_ratio_)}")
print("\n🔍 Hauptkomponente 1 Gewichte:")
print(f"   Jahresgehalt: {pca_roh.components_[0][0]:.6f}")
print(f"   Leistungsbewertung: {pca_roh.components_[0][1]:.6f}")
print("\n🔍 Hauptkomponente 2 Gewichte:")
print(f"   Jahresgehalt: {pca_roh.components_[1][0]:.6f}")
print(f"   Leistungsbewertung: {pca_roh.components_[1][1]:.6f}")

print("\n❌ PROBLEME:")
print(f"• HK1 erklärt {pca_roh.explained_variance_ratio_[0]:.1%} - fast alles!")
print(f"• HK2 erklärt nur {pca_roh.explained_variance_ratio_[1]:.1%}")
print(f"• Leistungsbewertungs-Gewicht ist winzig: {abs(pca_roh.components_[0][1]):.6f}")
print(f"• PCA ignoriert im Wesentlichen die Leistung und sieht nur Gehalt!")

In [ ]:
# Visualisiere PCA ohne Standardisierung
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Originaldaten mit Hauptkomponenten-Richtungen
ax1.scatter(X[:, 0], X[:, 1], s=100, alpha=0.7, c='steelblue')

# Füge Hauptkomponenten-Pfeile hinzu
mittelpunkt = np.mean(X, axis=0)
hk1_pfeil = pca_roh.components_[0] * 15000  # Skaliere für Sichtbarkeit
hk2_pfeil = pca_roh.components_[1] * 8000

ax1.arrow(mittelpunkt[0], mittelpunkt[1], hk1_pfeil[0], hk1_pfeil[1], 
          head_width=1500, head_length=2000, fc='red', ec='red', linewidth=3,
          label=f'HK1 ({pca_roh.explained_variance_ratio_[0]:.1%} Varianz)')
ax1.arrow(mittelpunkt[0], mittelpunkt[1], hk2_pfeil[0], hk2_pfeil[1], 
          head_width=1500, head_length=2000, fc='blue', ec='blue', linewidth=3,
          label=f'HK2 ({pca_roh.explained_variance_ratio_[1]:.1%} Varianz)')

ax1.set_xlabel('Jahresgehalt (€)')
ax1.set_ylabel('Leistungsbewertung (1-5)')
ax1.set_title('Rohdaten mit Hauptkomponenten\n(HK1 dominiert durch Gehaltsskala)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Daten im HK-Raum
farben = ['green' if leistung > 4.5 else 'red' if leistung < 2.5 else 'orange' 
          for leistung in df['Leistungsbewertung']]
ax2.scatter(X_pca_roh[:, 0], X_pca_roh[:, 1], s=100, alpha=0.7, c=farben)
ax2.set_xlabel(f'HK1 ({pca_roh.explained_variance_ratio_[0]:.1%} Varianz)')
ax2.set_ylabel(f'HK2 ({pca_roh.explained_variance_ratio_[1]:.1%} Varianz)')
ax2.set_title('Mitarbeiter im Hauptkomponenten-Raum\n(Grün=Hochleister, Rot=Niedrigleister)')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax2.axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("🤔 Bemerke: Der HK-Raum trennt nicht klar zwischen Hoch- und Niedrigleistern!")
print("Die Gehaltsskala dominiert und versteckt die Leistungsmuster.")

## 5. PCA mit Standardisierung

Jetzt lass uns die Merkmale standardisieren und den dramatischen Unterschied sehen:

In [ ]:
# Standardisiere die Daten
skalierer = StandardScaler()
X_skaliert = skalierer.fit_transform(X)

# Wende PCA auf standardisierte Daten an
pca_skaliert = PCA()
X_pca_skaliert = pca_skaliert.fit_transform(X_skaliert)

print("✅ PCA MIT STANDARDISIERUNG")
print("=" * 45)
print(f"📊 Erklärtes Varianzverhältnis: {pca_skaliert.explained_variance_ratio_}")
print(f"📈 Kumulative Varianz: {np.cumsum(pca_skaliert.explained_variance_ratio_)}")
print("\n🔍 Hauptkomponente 1 Gewichte:")
print(f"   Jahresgehalt: {pca_skaliert.components_[0][0]:.3f}")
print(f"   Leistungsbewertung: {pca_skaliert.components_[0][1]:.3f}")
print("\n🔍 Hauptkomponente 2 Gewichte:")
print(f"   Jahresgehalt: {pca_skaliert.components_[1][0]:.3f}")
print(f"   Leistungsbewertung: {pca_skaliert.components_[1][1]:.3f}")

print("\n✨ VERBESSERUNGEN:")
print(f"• HK1 erklärt jetzt {pca_skaliert.explained_variance_ratio_[0]:.1%} (war {pca_roh.explained_variance_ratio_[0]:.1%})")
print(f"• HK2 erklärt jetzt {pca_skaliert.explained_variance_ratio_[1]:.1%} (war {pca_roh.explained_variance_ratio_[1]:.1%})")
print(f"• Beide Merkmale tragen sinnvoll zu HK1 bei!")
print(f"• Leistungsbewertungs-Gewicht: {abs(pca_skaliert.components_[0][1]):.3f} (war {abs(pca_roh.components_[0][1]):.6f})")

# Interpretiere die Geschäftsbedeutung
if pca_skaliert.components_[0][0] * pca_skaliert.components_[0][1] < 0:
    print(f"\n🎯 GESCHÄFTSEINBLICK: HK1 zeigt 'Vergütungseffizienz'")
    print(f"   Hohe HK1 = Hohes Gehalt, niedrige Leistung (überbezahlt)")
    print(f"   Niedrige HK1 = Niedriges Gehalt, hohe Leistung (unterbezahlte Aufsteiger)")
else:
    print(f"\n🎯 GESCHÄFTSEINBLICK: HK1 zeigt 'Gesamten Mitarbeiterwert'")
    print(f"   Hohe HK1 = Hohes Gehalt, hohe Leistung")
    print(f"   Niedrige HK1 = Niedriges Gehalt, niedrige Leistung")

In [ ]:
# Zeige standardisierte Datenstatistiken
print("📏 STANDARDISIERTE DATENCHARAKTERISTIKA:")
print("=" * 45)
df_skaliert = pd.DataFrame(X_skaliert, columns=['Gehalt_Standardisiert', 'Leistung_Standardisiert'])
print(df_skaliert.describe())
print("\n✅ Beide Merkmale haben jetzt Mittelwert ≈ 0 und Std ≈ 1")
print("✅ Gleiche Skalen ermöglichen fairen Vergleich in PCA")

In [ ]:
# Visualisiere PCA mit Standardisierung
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Standardisierte Daten mit Hauptkomponenten
ax1.scatter(X_skaliert[:, 0], X_skaliert[:, 1], s=100, alpha=0.7, c='steelblue')

# Füge Hauptkomponenten-Pfeile hinzu
mittel_skaliert = np.mean(X_skaliert, axis=0)
hk1_pfeil_skaliert = pca_skaliert.components_[0] * 2  # Skaliere für Sichtbarkeit
hk2_pfeil_skaliert = pca_skaliert.components_[1] * 2

ax1.arrow(mittel_skaliert[0], mittel_skaliert[1], hk1_pfeil_skaliert[0], hk1_pfeil_skaliert[1], 
          head_width=0.15, head_length=0.2, fc='red', ec='red', linewidth=3,
          label=f'HK1 ({pca_skaliert.explained_variance_ratio_[0]:.1%} Varianz)')
ax1.arrow(mittel_skaliert[0], mittel_skaliert[1], hk2_pfeil_skaliert[0], hk2_pfeil_skaliert[1], 
          head_width=0.15, head_length=0.2, fc='blue', ec='blue', linewidth=3,
          label=f'HK2 ({pca_skaliert.explained_variance_ratio_[1]:.1%} Varianz)')

ax1.set_xlabel('Jahresgehalt (standardisiert)')
ax1.set_ylabel('Leistungsbewertung (standardisiert)')
ax1.set_title('Standardisierte Daten mit Hauptkomponenten\n(Beide Merkmale tragen sinnvoll bei)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Daten im HK-Raum (gefärbt nach Leistung)
farben = ['green' if leistung > 4.5 else 'red' if leistung < 2.5 else 'orange' 
          for leistung in df['Leistungsbewertung']]
ax2.scatter(X_pca_skaliert[:, 0], X_pca_skaliert[:, 1], s=100, alpha=0.7, c=farben)
ax2.set_xlabel(f'HK1 ({pca_skaliert.explained_variance_ratio_[0]:.1%} Varianz)')
ax2.set_ylabel(f'HK2 ({pca_skaliert.explained_variance_ratio_[1]:.1%} Varianz)')
ax2.set_title('Mitarbeiter im HK-Raum (Standardisiert)\n(Bessere Trennung von Mitarbeitertypen!)')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax2.axvline(x=0, color='k', linestyle='--', alpha=0.3)

# Füge Legende für Farben hinzu
from matplotlib.patches import Patch
legende_elemente = [Patch(facecolor='green', label='Hochleister (>4,5)'),
                   Patch(facecolor='orange', label='Durchschnittsleister'),
                   Patch(facecolor='red', label='Niedrigleister (<2,5)')]
ax2.legend(handles=legende_elemente, loc='upper right')

plt.tight_layout()
plt.show()

print("🎉 Viel besser! Jetzt können wir sinnvolle Mitarbeitersegmente im HK-Raum sehen!")

## 6. Dramatischer Vergleich

In [ ]:
# Erstelle umfassenden Vergleich
vergleichsdaten = {
    'Kennzahl': [
        'HK1 Erklärte Varianz',
        'HK2 Erklärte Varianz',
        'HK1 Gehalts-Gewicht',
        'HK1 Leistungs-Gewicht',
        'HK2 Gehalts-Gewicht', 
        'HK2 Leistungs-Gewicht'
    ],
    'Ohne Standardisierung': [
        f"{pca_roh.explained_variance_ratio_[0]:.3f}",
        f"{pca_roh.explained_variance_ratio_[1]:.3f}",
        f"{pca_roh.components_[0][0]:.6f}",
        f"{pca_roh.components_[0][1]:.6f}",
        f"{pca_roh.components_[1][0]:.6f}",
        f"{pca_roh.components_[1][1]:.6f}"
    ],
    'Mit Standardisierung': [
        f"{pca_skaliert.explained_variance_ratio_[0]:.3f}",
        f"{pca_skaliert.explained_variance_ratio_[1]:.3f}",
        f"{pca_skaliert.components_[0][0]:.3f}",
        f"{pca_skaliert.components_[0][1]:.3f}",
        f"{pca_skaliert.components_[1][0]:.3f}",
        f"{pca_skaliert.components_[1][1]:.3f}"
    ]
}

vergleichs_df = pd.DataFrame(vergleichsdaten)
print("📊 DETAILLIERTER VERGLEICH")
print("=" * 60)
print(vergleichs_df.to_string(index=False))

# Berechne Verbesserungen
hk2_verbesserung = (pca_skaliert.explained_variance_ratio_[1] - pca_roh.explained_variance_ratio_[1]) * 100
leistungsgewicht_verbesserung = abs(pca_skaliert.components_[0][1]) / abs(pca_roh.components_[0][1])

print(f"\n🚀 DRAMATISCHE VERBESSERUNGEN:")
print(f"• HK2-Varianz stieg um {hk2_verbesserung:.1f} Prozentpunkte!")
print(f"• Leistungsgewicht in HK1 stieg um das {leistungsgewicht_verbesserung:.0f}-fache!")
print(f"• Jetzt tragen beide Merkmale sinnvoll zur Analyse bei")

In [ ]:
# Visueller Vergleich der erklärten Varianz
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Ohne Standardisierung
komponenten = ['HK1', 'HK2']
varianz_roh = pca_roh.explained_variance_ratio_
bars1 = ax1.bar(komponenten, varianz_roh, color=['red', 'blue'], alpha=0.7)
ax1.set_title('Ohne Standardisierung\n(Gehalt dominiert)')
ax1.set_ylabel('Erklärtes Varianzverhältnis')
ax1.set_ylim(0, 1)
for i, v in enumerate(varianz_roh):
    ax1.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=12)

# Plot 2: Mit Standardisierung
varianz_skaliert = pca_skaliert.explained_variance_ratio_
bars2 = ax2.bar(komponenten, varianz_skaliert, color=['red', 'blue'], alpha=0.7)
ax2.set_title('Mit Standardisierung\n(Ausgewogener Beitrag)')
ax2.set_ylabel('Erklärtes Varianzverhältnis')
ax2.set_ylim(0, 1)
for i, v in enumerate(varianz_skaliert):
    ax2.text(i, v + 0.02, f'{v:.1%}', ha='center', fontweight='bold', fontsize=12)

# Plot 3: Verbesserungsvergleich
hk_labels = ['HK1', 'HK2']
x = np.arange(len(hk_labels))
width = 0.35

ax3.bar(x - width/2, varianz_roh, width, label='Ohne Standardisierung', 
        color='lightcoral', alpha=0.7)
ax3.bar(x + width/2, varianz_skaliert, width, label='Mit Standardisierung', 
        color='lightgreen', alpha=0.7)

ax3.set_ylabel('Erklärtes Varianzverhältnis')
ax3.set_title('Dramatische Verbesserung!')
ax3.set_xticks(x)
ax3.set_xticklabels(hk_labels)
ax3.legend()
ax3.set_ylim(0, 1)

# Hebe die Verbesserung hervor
verbesserungs_pfeil = ax3.annotate('', xy=(1 + width/2, varianz_skaliert[1]), 
                                xytext=(1 - width/2, varianz_roh[1]),
                                arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax3.text(1, (varianz_roh[1] + varianz_skaliert[1])/2, 
         f'+{hk2_verbesserung:.1f}pp', ha='center', va='center', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
         fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Geschäftsinterpretation

In [ ]:
# Interpretiere die Ergebnisse in Geschäftsbegriffen
print("💼 GESCHÄFTSEINBLICKE AUS PCA")
print("=" * 50)

# Analysiere Mitarbeitersegmente im HK-Raum
hk1_werte = X_pca_skaliert[:, 0]
hk2_werte = X_pca_skaliert[:, 1]

# Erstelle Mitarbeitersegmente
segmente = []
for i, (hk1, hk2, gehalt, leistung) in enumerate(zip(hk1_werte, hk2_werte, 
                                                  df['Jahresgehalt'], df['Leistungsbewertung'])):
    if hk1 > 0.5 and leistung < 3.0:
        segment = "🔴 Überbezahlt (Überprüfung nötig)"
    elif hk1 < -0.5 and leistung > 4.0:
        segment = "🟢 Aufsteiger (Befördern/Erhöhen)"
    elif hk1 > 0 and leistung > 4.0:
        segment = "⭐ Hochleister (Halten)"
    elif hk1 < 0 and leistung < 3.0:
        segment = "🟡 Niedrigleister (Schulen/PIP)"
    else:
        segment = "🔵 Ausgewogen (Standard)"
    
    segmente.append(segment)

# Füge Segmente zum Dataframe hinzu
df_analyse = df.copy()
df_analyse['HK1_Wert'] = hk1_werte.round(2)
df_analyse['HK2_Wert'] = hk2_werte.round(2)
df_analyse['HR_Segment'] = segmente

print("👥 MITARBEITERSEGMENTIERUNG:")
print(df_analyse[['Mitarbeiter_ID', 'Jahresgehalt', 'Leistungsbewertung', 'HR_Segment']].to_string(index=False))

# Zähle Segmente
segment_zaehlung = df_analyse['HR_Segment'].value_counts()
print(f"\n📊 SEGMENTVERTEILUNG:")
for segment, anzahl in segment_zaehlung.items():
    print(f"{segment}: {anzahl} Mitarbeiter")

# Geschäftsempfehlungen
print(f"\n🎯 GESCHÄFTSEMPFEHLUNGEN:")
print(f"1. 🟢 Aufsteiger: Sofortige Gehaltserhöhungen erwägen")
print(f"2. 🔴 Überbezahlte Mitarbeiter: Leistungsverbesserungspläne")
print(f"3. ⭐ Hochleister: Fokus auf Bindungsstrategien")
print(f"4. 🟡 Niedrigleister: Schulung oder Leistungsmanagement")
print(f"5. 🔵 Ausgewogene Mitarbeiter: Standard-Karriereentwicklung")

## 8. Wichtige Erkenntnisse

In [ ]:
print("🎓 WESENTLICHE LEKTIONEN GELERNT")
print("=" * 50)
print()
print("1. 📏 SKALA IST ABSOLUT WICHTIG:")
print(f"   • Ohne Standardisierung: HK2 erklärte nur {pca_roh.explained_variance_ratio_[1]:.1%}")
print(f"   • Mit Standardisierung: HK2 erklärt {pca_skaliert.explained_variance_ratio_[1]:.1%}")
print(f"   • Das sind {hk2_verbesserung:.1f} Prozentpunkte Verbesserung!")
print()
print("2. 🔧 STANDARDISIERUNG OFFENBART VERBORGENE MUSTER:")
print("   • Rohdaten: Sah nur Gehaltsunterschiede")
print("   • Standardisiert: Entdeckte Vergütungseffizienz-Muster")
print("   • Ermöglichte sinnvolle Mitarbeitersegmentierung")
print()
print("3. 💼 GESCHÄFTSWERT:")
print("   • Identifizierte überbezahlte Niedrigleister")
print("   • Fand unterbezahlte Hochleister (Kündigungsrisiko)")
print("   • Schuf umsetzbare HR-Segmente")
print()
print("4. 🎯 WANN STANDARDISIEREN:")
print("   ✅ Verschiedene Einheiten (Euro vs. Bewertungen)")
print("   ✅ Verschiedene Skalen (Tausende vs. Einzelziffern)")
print("   ✅ Alle Merkmale sollen gleich beitragen")
print("   ❌ Skalenunterschiede sind bedeutsam")
print("   ❌ Merkmale bereits auf ähnlichen Skalen")
print()
print("5. 🚀 ANWENDUNG IN DER PRAXIS:")
print("   • Dieses 2D-Beispiel skaliert auf 20+ Dimensionen")
print("   • Gleiche Prinzipien gelten für Kundensegmentierung")
print("   • Kritisch für jede multivariable Analyse")

print("\n🏆 FAZIT:")
print("Standardisierung ist nicht nur ein technischer Schritt - es ist der Schlüssel zum")
print("Entdecken sinnvoller Geschäftseinblicke, die Entscheidungen antreiben!")

## 9. Übung: Probiere es selbst!

In [ ]:
# 🎮 PRAKTISCHE ÜBUNG
print("🎮 DU BIST DRAN: DATEN MODIFIZIEREN")
print("=" * 45)
print("Versuche verschiedene Mitarbeiterszenarien zu erstellen und zu sehen, wie PCA reagiert:")
print()
print("💡 EXPERIMENTIDEEN:")
print("1. Alle Mitarbeiter fair bezahlt (positive Korrelation)")
print("2. Extreme Gehaltsunterschiede (größeres Skalenproblem)")
print("3. Zufällige Gehaltszuweisung (keine Korrelation)")
print("4. Sehr enger Leistungsbereich (2D-Problem)")

# Beispiel-Übungsdaten - modifiziere diese!
uebungsdaten = np.array([
    # Modifiziere diese Werte und führe die PCA-Analyse erneut aus!
    # [Gehalt, Leistung]
    [45000, 4.8],   # Versuche diese Werte zu ändern
    [95000, 2.1],   # Was wäre wenn das [95000, 4.8] wäre?
    [48000, 4.9],   # Was ist mit extremen Gehältern?
    [88000, 2.5],   # Oder sehr ähnlichen Leistungsbewertungen?
    # Füge hier mehr Mitarbeiter hinzu...
])

print("\n🔧 ZUM EXPERIMENTIEREN:")
print("1. Modifiziere das uebungsdaten Array oben")
print("2. Führe PCA mit und ohne Standardisierung aus")
print("3. Vergleiche die erklärten Varianzverhältnisse")
print("4. Denke über die Geschäftsimplikationen nach")

print("\n💻 HERAUSFORDERUNG:")
print("Kannst du Daten erstellen, wo Standardisierung einen noch größeren Unterschied macht?")

## 10. Nächste Schritte & Echte Anwendungen

In [ ]:
print("🚀 NÄCHSTE SCHRITTE IN DEINER PCA-REISE")
print("=" * 50)
print()
print("📚 FORTGESCHRITTENE THEMEN ZUM ERKUNDEN:")
print("• Optimale Anzahl von Komponenten wählen (Scree-Plots, kumulative Varianz)")
print("• Andere Skalierungsmethoden (MinMaxScaler, RobustScaler)")
print("• PCA für Datenvisualisierung (t-SNE, UMAP Alternativen)")
print("• Loadings im Geschäftskontext interpretieren")
print("• PCA für Anomalieerkennung")
print()
print("🏢 ANWENDUNGEN IN DER PRAXIS:")
print("• Kundensegmentierung (Demografie + Verhalten)")
print("• Finanzportfolio-Optimierung")
print("• Bildkompression und Computer Vision")
print("• Genexpressionsanalyse in der Bioinformatik")
print("• Qualitätskontrolle in der Fertigung")
print("• Marktforschung und Umfrageanalyse")
print()
print("💻 ÜBUNGSDATENSÄTZE:")
print("• Iris-Datensatz (klassisches 4D-Beispiel)")
print("• Weinqualitäts-Datensatz")
print("• Boston Immobilienpreise")
print("• Kundensegmentierungs-Datensätze")
print("• Die Daten deines eigenen Unternehmens!")
print()
print("🎯 DENKE DARAN:")
print("Die Konzepte, die du hier mit 2 Variablen gelernt hast,")
print("skalieren direkt auf 20, 200 oder 2000 Variablen!")

## Zusammenfassung

In diesem Notebook haben wir eine **dramatische Demonstration** gesehen, warum Merkmalsstandardisierung für PCA entscheidend ist:

### Das Problem
- 📊 **Skalenunterschiede** können PCA-Ergebnisse völlig verzerren
- 💰 **Gehaltsdaten** (Tausende) dominierten **Leistungsbewertungen** (1-5 Skala)
- 🔍 **Verborgene Muster** blieben aufgrund von Skalenverzerrung unsichtbar

### Die Lösung
- ⚖️ **Standardisierung** gab allen Merkmalen gleiches Gewicht
- 📈 **HK2-Varianz** verbesserte sich von 0,4% auf 45% - ein **44,6 Prozentpunkte Anstieg**!
- 🎯 **Geschäftseinblicke** entstanden: Vergütungseffizienz-Muster

### Die Geschäftsauswirkung
- 🟢 **Aufsteiger identifiziert** (unterbezahlte Hochleister)
- 🔴 **Überbezahlte Niedrigleister gefunden**, die Aufmerksamkeit brauchen
- 💼 **Umsetzbare HR-Segmente** für strategische Entscheidungen erstellt

### Wichtigste Erkenntnis
**Standardisierung ist nicht nur ein Vorverarbeitungsschritt** - es ist der Schlüssel zum Freischalten sinnvoller Muster in deinen Daten, die echten Geschäftswert schaffen!

---

*Denke daran: Dieses 2D-Beispiel lehrt Konzepte, die auf jede Anzahl von Dimensionen skalieren. Ob du 2 Variablen oder 200 analysierst, die Wichtigkeit der richtigen Skalierung bleibt dieselbe.*